In [ ]:
import torch
import treescope
from models.gan import DinoPatchDiscriminator
from torchmetrics.image import (
    StructuralSimilarityIndexMeasure,
    PeakSignalNoiseRatio,
    FrechetInceptionDistance,
    LearnedPerceptualImagePatchSimilarity,
)

treescope.register_as_default()

In [ ]:
model = DinoPatchDiscriminator('small')
model(torch.randn(1, 3, 256, 256).cuda())

In [ ]:
from layers.layers2d import Encoder, Decoder
from models.vae import VAE

In [ ]:
vae = VAE(
    in_channels=3,
    out_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    dropout=0.0,
    resolution=256,
    z_channels=4,
    spatial_compression=4,
    wavelet="db4",
    maxlevel=2
)
vae

In [ ]:
vae.forward(torch.randn(1, 3, 256, 256))

In [ ]:
encoder = Encoder(
    in_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    z_channels=4,
    spatial_compression=8,
    dropout=0.1,
)

decoder = Decoder(
    z_channels=4,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    out_channels=3,
    spatial_compression=8,
    dropout=0.1,
)

In [ ]:
encoder

In [ ]:
decoder

In [ ]:
decoder.forward(torch.randn(1, 4, 8, 8)).shape

In [1]:
import torch
import einops
import treescope

from models.vqvae import VQVAE
from layers.quantizers import FSQuantizer
treescope.register_as_default()
treescope.basic_interactive_setup(autovisualize_arrays=True)


In [19]:
a = torch.randn((1, 1, 8))
b = torch.randn((1, 2, 8))

print(einops.rearrange(a, "... c d -> ... (c d)").shape)
print(einops.rearrange(b, "... c d -> ... (c d)").shape)

torch.Size([1, 8])
torch.Size([1, 16])


In [20]:
quantizer = FSQuantizer(levels=[7, 5, 5, 5], input_dim=256, num_codebooks=1)

In [21]:
codes, indices = quantizer(torch.randn(1, 256, 8, 8))

In [5]:
proj_codes, codes = quantizer.indices_to_codes(indices)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
vq = VQVAE(
    in_channels=3,
    out_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    dropout=0.0,
    resolution=256,
    z_channels=256,
    spatial_compression=4,
    wavelet="db4",
    maxlevel=2,
    quantization="fsq",
    levels=[8, 5, 5, 5],
    embedding_dim=4,
    num_codebooks=1,
)

In [ ]:
vq

In [ ]:
vq(torch.randn(1, 3, 256, 256))

In [26]:
import torch
from layers.quantizers import FSQuantizer, round_ste, round_rot
from einops import rearrange, einsum

In [27]:
quantizer = FSQuantizer(levels=[7, 5, 5, 5], input_dim=4, num_codebooks=1)

In [28]:
x = torch.randn(1, 8, 1, 4)
codes = quantizer.quantize(x)
indices = quantizer.codes_to_indices(codes)

In [29]:
codes.squeeze()

tensor([[ 0.0000,  0.5000,  0.0000,  1.0000],
        [ 0.0000, -0.5000, -1.0000, -0.5000],
        [-0.6667,  0.5000, -0.5000, -1.0000],
        [ 0.6667,  1.0000, -0.5000, -0.5000],
        [ 1.0000,  0.0000,  0.5000, -0.5000],
        [-1.0000,  1.0000,  0.5000,  0.5000],
        [ 0.3333, -0.5000,  0.5000,  0.5000],
        [-0.3333,  0.0000, -0.5000,  0.5000]])

In [30]:
a = indices.squeeze()

In [31]:
quantizer.rotate = True
codes = quantizer.quantize(x)
indices = quantizer.codes_to_indices(codes)

In [32]:
codes.squeeze()

tensor([[ 0.0000,  0.5000,  0.0000,  1.0000],
        [ 0.0000, -0.5000, -1.0000, -0.5000],
        [-0.6667,  0.5000, -0.5000, -1.0000],
        [ 0.6667,  1.0000, -0.5000, -0.5000],
        [ 1.0000,  0.0000,  0.5000, -0.5000],
        [-1.0000,  1.0000,  0.5000,  0.5000],
        [ 0.3333, -0.5000,  0.5000,  0.5000],
        [-0.3333,  0.0000, -0.5000,  0.5000]])

In [33]:
b = indices.squeeze()

In [34]:
a

tensor([794, 185,  57, 243, 300, 658, 641, 576], dtype=torch.int32)

In [35]:
b

tensor([794, 184,  56, 243, 300, 657, 641, 575], dtype=torch.int32)

In [36]:
a == b

tensor([ True, False, False,  True,  True, False,  True, False])